In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score ,f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.utils import resample
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

## 3.0 Now, let us load the required data into individual variables

In [6]:
X_train=pd.read_csv("C:/Users/surya/OneDrive/Documents/USF/Classes/Semester-2/DSP/Assignment-1/data/online_price_train_X.csv")
X_test = pd.read_csv("C:/Users/surya/OneDrive/Documents/USF/Classes/Semester-2/DSP/Assignment-1/data/online_price_test_X.csv")
y_train = pd.read_csv("C:/Users/surya/OneDrive/Documents/USF/Classes/Semester-2/DSP/Assignment-1/data/online_price_train_y.csv")
y_test = pd.read_csv("C:/Users/surya/OneDrive/Documents/USF/Classes/Semester-2/DSP/Assignment-1/data/online_price_test_y.csv")

Let us create a data frame called performance and put all the needed metrics in it!

In [7]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})

# 4.1.1 Logistic Regression with Random Search

In [8]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'penalty': ['None','l1','l2','elasticnet'],
    'solver':['saga','liblinear']
}

model = LogisticRegression()
rand_search = RandomizedSearchCV(estimator = model, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1, 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 8 is smaller than n_iter=500. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 8 candidates, totalling 40 fits


C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
20 fits failed out of a total of 40.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packa

The best recall score is 0.8746279620935514
... with parameters: {'solver': 'saga', 'penalty': 'l1'}


C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


# 4.1.2 Logistic Regression with Grid Search

In [9]:
score_measure = "recall"
kfolds = 5
penalty = rand_search.best_params_['penalty']
solver = rand_search.best_params_['solver']

param_grid = {  
    'penalty': [penalty],
    'solver': [solver]
}

model = LogisticRegression()
grid_search = GridSearchCV(estimator = model, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallLogistic = grid_search.best_estimator_

Fitting 5 folds for each of 1 candidates, totalling 5 fits


C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The best recall score is 0.8746111454787361
... with parameters: {'penalty': 'l1', 'solver': 'saga'}


C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [10]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Logistic Regression",
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,Logistic Regression,0.867494,0.866656,0.873732,0.87018


## 4.2.1 SVM (Linear, RBF, Poly) - Random Search

In [11]:
score_measure = "recall"
kfolds = 2

param_grid = {
    'C': np.arange(1,20),   
    'gamma': ['scale','auto'],
    'kernel':['linear','rbf','poly']
}

model = SVC()
rand_search = RandomizedSearchCV(estimator = model, param_distributions=param_grid, cv=kfolds, n_iter=100,
                           scoring=score_measure, verbose=1, n_jobs=-1, 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

Fitting 2 folds for each of 100 candidates, totalling 200 fits


## 4.2.2 SVM (Linear, RBF, Poly) - Grid Search

In [ ]:
score_measure = "recall"
kfolds = 5

C = rand_search.best_params_['C']
gamma = rand_search.best_params_['gamma']
kernel = rand_search.best_params_['kernel']

param_grid = {
    'C': np.arange(C-3,C+3),  
    'gamma': [gamma],
    'kernel': [kernel]
    
}

model = SVC()
grid_search = GridSearchCV(estimator = model, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallSVM = grid_search.best_estimator_

Fitting 5 folds for each of 6 candidates, totalling 30 fits
The best recall score is 1.0
... with parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}


C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"SVM", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,Logistic Regression,0.910256,0.932584,0.912088,0.922222
0,SVM,0.583333,0.583333,1.000000,0.736842


## 4.3.1 Decision Trees - Random Search 

In [ ]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(2,100),  
    'min_samples_leaf': np.arange(1,75),
    'min_impurity_decrease': np.arange(0.0001, 0.01, 0.0005),
    'max_leaf_nodes': np.arange(5, 80), 
    'max_depth': np.arange(1,40), 
    'criterion': ['entropy', 'gini'],
}

model = DecisionTreeClassifier()
rand_search = RandomizedSearchCV(estimator = model, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1, 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best recall score is 0.9521739130434783
... with parameters: {'min_samples_split': 88, 'min_samples_leaf': 8, 'min_impurity_decrease': 0.0006000000000000001, 'max_leaf_nodes': 48, 'max_depth': 39, 'criterion': 'entropy'}


## 4.3.2 DECISION TREE - GRID SEARCH

In [ ]:
score_measure = "recall"
kfolds = 5
min_samples_split = rand_search.best_params_['min_samples_split']
min_samples_leaf = rand_search.best_params_['min_samples_leaf']
min_impurity_decrease = rand_search.best_params_['min_impurity_decrease']
max_leaf_nodes = rand_search.best_params_['max_leaf_nodes']
max_depth = rand_search.best_params_['max_depth']
criterion = rand_search.best_params_['criterion']

param_grid = {
    'min_samples_split': np.arange(min_samples_split-2,min_samples_split+2),  
    'min_samples_leaf': np.arange(min_samples_leaf-2,min_samples_leaf+2),
    'min_impurity_decrease': np.arange(min_impurity_decrease-0.0001, min_impurity_decrease+0.0001, 0.00005),
    'max_leaf_nodes': np.arange(max_leaf_nodes-2,max_leaf_nodes+2), 
    'max_depth': np.arange(max_depth-2,max_depth+2), 
    'criterion': [criterion]
}

model = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = model, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

Fitting 5 folds for each of 1280 candidates, totalling 6400 fits
The best recall score is 0.9521739130434783
... with parameters: {'criterion': 'entropy', 'max_depth': 37, 'max_leaf_nodes': 46, 'min_impurity_decrease': 0.0005, 'min_samples_leaf': 6, 'min_samples_split': 86}


In [ ]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Decision Tree", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

In [ ]:
performance

,model,Accuracy,Precision,Recall,F1
0,Logistic Regression,0.910256,0.932584,0.912088,0.922222
0,SVM,0.583333,0.583333,1.000000,0.736842
0,Decision Tree,0.903846,0.872549,0.978022,0.922280
